# University Cost RAG Pipeline

A self‑contained Colab/Notebook to chat over the `AISC_Cleaned.csv` dataset using a hybrid approach:

* **Pandas / SQL** for structured numeric look‑ups (tuition comparisons, filters, aggregates).
* **OpenAI GPT‑4o (or any chat‑model)** to turn user questions into Pandas code, run it, then answer in plain English.

> *Adapted & simplified from the original HF‑Embedding RAG cookbook.*

## 0. Install dependencies
Skip this cell if you already have the packages in your environment.

In [30]:
!pip install --quiet pandas openai
!pip install --quiet anthropic
!pip install --quiet faiss-cpu
!pip install --quiet transformers==4.41.0
!pip install --quiet -U sentence-transformers

In [31]:
import json, os, re, textwrap, pickle, pathlib, sys
import pandas as pd #library for dataframes
import tiktoken #library to estimate tokens used for each character
import faiss
from anthropic import Anthropic
import numpy as np


## 1. Imports & API key
Replace `YOUR_OPENAI_KEY` with your real key or load it from an `.env`.

In [32]:
ANTHROPIC_API_KEY = "XXXXX"
client_ant = Anthropic(
    api_key=ANTHROPIC_API_KEY,
)

## 2. Load the CSV

In [33]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [34]:
csv_path = "/content/drive/My Drive/Colab Notebooks/AISC/groups/AISC_Cleaned.csv"  # if running in Colab, upload first or mount Drive
df = pd.read_csv(csv_path, encoding='latin1')
print(f"Loaded {len(df):,} rows and {df.shape[1]} columns.")
df.head(1)

Loaded 306 rows and 21 columns.


,UNITID,OPEID,OPEID6,INSTNM,CITY,STABBR,ZIP,ACCREDAGENCY,INSTURL,NPCURL,REGION,LOCALE,LATITUDE,SAT_AVG,BOOKSUPPLY,ROOMBOARD_ON,OTHEREXPENSE_ON,ROOMBOARD_OFF,OTHEREXPENSE_OFF,SCORECARD_SECTOR,GRANTS OPPS
0,110486,799300,7993,California State University-Bakersfield,Bakersfield,CA,93311-1099,Western Association of Schools and Colleges Se...,www.csub.edu/,https://www2.calstate.edu/attend/paying-for-co...,8,11,35.350005,NaN,1068.0,14430.0,3782.0,16316.0,6424.0,4,Cal Grant


## 3. Create a Pandas‑aware agent
The agent can execute Pandas code for us, then craft a friendly answer.

In [62]:
SCHEMA = {
  "type": "object",
  "properties": {
    "UNITID": {
      "type": "integer",
      "description": "IPEDS unique institutional identifier"
    },
    "OPEID": {
      "type": "integer",
      "description": "8-digit U.S. Dept. of Education Office of Postsecondary Education ID"
    },
    "OPEID6": {
      "type": "integer",
      "description": "6-digit base OPEID (institutional component)"
    },
    "INSTNM": {
      "type": "string",
      "description": "Official institution name"
    },
    "CITY": {
      "type": "string",
      "description": "City where the main campus is located"
    },
    "STABBR": {
      "type": "string",
      "description": "U.S. state abbreviation (e.g., CA, TX)"
    },
    "ZIP": {
      "type": "string",
      "description": "ZIP or ZIP+4 postal code"
    },
    "ACCREDAGENCY": {
      "type": "string",
      "description": "Primary institutional accrediting agency"
    },
    "INSTURL": {
      "type": "string",
      "description": "Institution website URL"
    },
    "NPCURL": {
      "type": "string",
      "description": "Net-Price Calculator URL"
    },
    "REGION": {
      "type": "integer",
      "description": "IPEDS geographic region code (1–8)"
    },
    "LOCALE": {
      "type": "integer",
      "description": "Locale/urbanization code (11=City, 13=Suburb, etc.)"
    },
    "LATITUDE": {
      "type": "number",
      "description": "Campus latitude in decimal degrees"
    },
    "SAT_AVG": {
      "type": "number",
      "description": "Average SAT composite score of enrolled freshmen"
    },
    "BOOKSUPPLY": {
      "type": "number",
      "description": "Estimated annual cost of books and supplies (USD)"
    },
    "ROOMBOARD_ON": {
      "type": "number",
      "description": "Annual on-campus room & board cost (USD)"
    },
    "OTHEREXPENSE_ON": {
      "type": "number",
      "description": "Other on-campus expenses (transportation, personal, etc.) in USD"
    },
    "ROOMBOARD_OFF": {
      "type": "number",
      "description": "Annual off-campus room & board cost (USD)"
    },
    "OTHEREXPENSE_OFF": {
      "type": "number",
      "description": "Other off-campus expenses in USD"
    },
    "SCORECARD_SECTOR": {
      "type": "integer",
      "description": "College Scorecard sector code (public, private-nonprofit, etc.)"
    },
    "GRANTS OPPS": {
      "type": "string",
      "description": "Grant or opportunity program tags (e.g., “Cal Grant”)"
    }
  },
  "required": []
}


In [77]:
PANDAS_PROMPT = f"""
  You are an expert Python–Pandas data assistant.

  DATA AVAILABLE
  • A Pandas DataFrame named `df` is already in memory.
  • Schema (column → dtype):

  {SCHEMA}

  YOUR TASK, STEP-BY-STEP
  1. Understand the user’s question and decide which Pandas operations are needed
    (filter, sort, group-by, aggregate, etc.). Think through the logic *before*
    you write code.

  2. Write a concise Python snippet that:
      • uses only the `df` provided (do **not** read files or import libraries);
      • creates a string variable  **`answer`**  summarising the result
        in plain English (≤ 40 words, no markdown);
      • *optionally* creates a DataFrame **`table`** (≤ 10 rows, ≤ 12 columns)
        that backs up the answer. If no table is useful, omit it.

        Do **not** call `print()` or display anything;
        assign to variables only.  The host code will handle output.

  3. Return **only** a JSON object with exactly these keys:
      • `"code"`        – a string containing the Python you just wrote
      • `"explanation"` – one sentence that tells the user what you found

    Example:
    {{
      "code": "filtered = df[df['State']== 'CA']\\nlowest = filtered.nsmallest(3, 'TotalCost')\\nanswer = f'The three least-expensive CA schools are {{lowest[\"Name\"].tolist()}}.'\\ntable = lowest[['Name', 'TotalCost']]",
      "explanation": "Here are the three California universities with the lowest total cost."
    }}

  4. **Output nothing else.** No markdown fences, no commentary.

  REMINDERS
  * Prefer vectorised Pandas methods over loops.
  * Always reference column names exactly as in the schema.
  * If the question asks for a count/mean/min/max, include the numeric result
    in `answer`.
  * Keep `table` small and relevant; truncate with `.head()` if necessary.
"""


In [104]:
def run_code(code: str) -> tuple[str, pd.DataFrame | None]:
    """
    Execute LLM-generated code in a *very* thin sandbox.
    The code must create a variable `answer` (str) and, optionally, `table`
    (a small Pandas DataFrame).  Returns (answer, table_or_None).
    """
    local_ns = {"df": df.copy()}        # df should already be defined globally
    exec(code, {}, local_ns)            # ⚠️  evaluate with caution!
    return local_ns.get("answer", ""), local_ns.get("table")

# --- new wrapper ------------------------------------------------------------
def query_dataframe_agent(user_question: str):
    """
    • Sends the question to Claude-3 Sonnet with a schema-aware prompt.
    • Parses the JSON { "code": "...", "explanation": "..." } returned.
    • Runs the code and returns (explanation, answer, table).
    """
    # 1. Sends question to claude to generate code
    resp = client_ant.messages.create(
        model="claude-3-7-sonnet-20250219",
        system=PANDAS_PROMPT,
        temperature=0,
        # Added: max_tokens argument to satisfy the function requirements
        max_tokens=500,
        messages=[
            # Remove system prompt from messages
            {"role": "user",   "content": user_question},
        ],
    )

    # 2. Anthropic SDK → resp.content[0].text
    if hasattr(resp, "content"): # if the result has content
        raw_json = resp.content[0].text

    #Added: Attempt to repair truncated JSON before parsing
    if not raw_json.endswith("}"):
        raw_json += "}"

    try:
        payload = json.loads(raw_json) # the result from pandas agent
    except json.JSONDecodeError:
        raise ValueError(f"Model did not return valid JSON:\n{raw_json}")

    # 3. Return the result of the executed code
    code        = payload.get("code", "") #grab code logic from agent
    explanation = payload.get("explanation", "").strip() #grab explanation/reasoning of agent

    print('\n')
    print(f"REASONING GENRATED BY THE AGENT: '\n' {explanation}")

    print('\n')
    print(f"CODE GENRATED BY THE AGENT: '\n'  {code}")

    answer, table = run_code(code)             # execute the LLM’s Pandas snippet
    return explanation, answer, table

## 4. Create a Text‑aware agent
The agent will execute any information that relates with text data and numeric from the dataset

In [105]:
def university_context_agent():
  #---TODO---#
  # How to make this agent respond with text data from each university
  pass

## 5. Manager agent
The agent will route any query problem to the appropriate agent to complete the user query


In [106]:
def process_tool_call(tool_name: str, tool_input: dict) -> str:
    """
    Dispatches calls from the manager / router LLM to the right helper.

    Parameters
    ----------
    tool_name : str
        One of  {"dataframe_agent", "context_agent"}  (rename as you like).
    tool_input : dict
        Payload expected by the tool:
          • dataframe_agent → {"question": "..."}
          • context_agent   → {"product_name": "...", "feature": "..."}

    Returns
    -------
    str
        Human-readable text to feed back to the chat LLM.
    """

    # ─── 1. Structured numeric / Pandas path ────────────────────────────────
    if tool_name == "query_dataframe_agent":
        question = tool_input.get("question", "")
        if not question:
            return "❗️ query_dataframe_agent needs a 'question' field."

        explanation, answer, table = query_dataframe_agent(question)

        # Format the optional table nicely if it exists
        table_str = ""
        if table is not None and not table.empty:
            table_str = "\n" + table.to_markdown(index=False) + "\n"

        return f"{explanation}\n{table_str}{answer}"

    # ─── 2. Semantic / descriptive-text path ───────────────────────────────
    elif tool_name == "university_context_agent":
        # call university context agent here - ###------TODO------#
        pass

    # ─── 3. Fallback ───────────────────────────────────────────────────────
    return f"❓ Tool '{tool_name}' not found."


## 6. Try it out!

In [107]:
# ── Tool definitions for Claude “function calling” ─────────────────────────
tools = [
    {
        "name": "query_dataframe_agent",
        "description": (
            "Run Pandas over the universities DataFrame to answer numeric or "
            "logic-based questions (filtering, sorting, min/max, averages, etc.). "
            "Call this when precise calculations on tuition, fees, room-and-board, "
            "or any other structured column are required."
        ),
        "input_schema": {
            "type": "object",
            "properties": {
                "question": {
                    "type": "string",
                    "description": (
                        "A natural-language question that the agent will convert "
                        "into Pandas code (e.g. "
                        "'Which three universities in California have the lowest "
                        "in-state total cost of attendance?')."
                    ),
                }
            },
            "required": ["question"],
        },
    },
    {
        "name": "university_context_agent",
        "description": (
            "Fetch qualitative or descriptive information about a university—"
            "mission statement, notable programs, campus culture, etc.—that "
            "is not contained in the numeric DataFrame.  Use when the user asks "
            "about attributes that require textual context rather than numbers."
        ),
        "input_schema": {
            "type": "object",
            "properties": {
                "product_name": {            # keep the names your code expects
                    "type": "string",
                    "description": "Name of the university to look up",
                },
                "feature": {
                    "type": "string",
                    "description": (
                        "The specific aspect to retrieve or compare "
                        "(e.g. 'sustainability focus', 'engineering reputation')."
                    ),
                },
            },
            "required": ["product_name", "feature"],
        },
    },
]


In [108]:
def generate_response_with_rag(query: str) -> str:
    """
    One-shot retrieval-augmented generation with optional tool calls.
    Supports two tools the model may invoke:
      • query_dataframe_agent   (structured Pandas/SQL path)
      • university_context_agent     (semantic / descriptive path)
    """

    system_prompt = f"""
        You are a helpful assistant who answers questions about US universities
        using both numeric data (tuition, fees, etc.) and descriptive text
        (mission statements, program blurbs).

        You have access to two callable tools:

          • query_dataframe_agent(question)      – run Pandas over the numeric CSV
          • university_context_agent(product_name, feature) – fallback semantic helper (rename as needed)

        When the user’s query can be answered *directly* from the context below,
        answer without calling a tool.
        If you need fresh numeric calculations or row filtering, call
        **query_dataframe_agent**.
        If you need qualitative details not in the numeric table, call
        **university_context_agent**.
      """

    follow_prompt = """
      You are wrapping up a retrieval-and-tool call cycle.

      You now have:
      •  the original user question:  «{user_question}»
      •  the tool you invoked:        «{tool_name}»
      •  the tool’s JSON result:
          - explanation  (plain English reasoning you wrote earlier)
          - code         (Pandas snippet you generated)
          - answer       (string from the executed code)
          - table        (optional DataFrame, ≤ 10 rows, already formatted)

      Compose a concise response with **three sections**:

      ────────────────────────────────────────────────────────
      **1. How I answered**
        A 2-sentence recap of the mental steps that led you to
        call this tool and pick those filters / aggregates.

      **2. Code in plain English**
        One short paragraph that describes what the Pandas
        code does (e.g. “Filter df to CA schools, sort by
        in-state cost, take the three smallest rows”).

      **3. Result**
        – Restate the final `answer` string.
        – If a table exists, show it in GitHub-style markdown
          (use the already formatted table text).
      ────────────────────────────────────────────────────────

      Don’t include the raw JSON or full code.
      Keep the whole response under ~180 words.
    """

    # 2️⃣  First model call  ───────────── Figure out which tool to use ─────────────
    first = client_ant.messages.create(
        model="claude-3-7-sonnet-20250219",
        system=system_prompt,
        messages=[{"role": "user", "content": query}],
        tools=tools,
        temperature=0.3,
        max_tokens=500,
    )

    # Helper to extract plaintext from an Anthropic message  ────────────────
    def plaintext(msg):
        return "".join(
            block.text for block in msg.content if getattr(block, "type", None) == "text"
        )

    # 3️⃣  If Claude decided to call a tool …  ──────────────────────────────
    if first.stop_reason == "tool_use":
        tool_block = next(b for b in first.content if b.type == "tool_use")
        tool_name  = tool_block.name
        tool_input = tool_block.input

        # run the backend helper
        tool_result = process_tool_call(tool_name, tool_input)

        # 4️⃣  Send the *tool_result* back to Claude  ──────────────────────
        follow = client_ant.messages.create(
            model="claude-3-7-sonnet-20250219",
            system=follow_prompt,
            messages=[
                {"role": "user",      "content": query},
                {"role": "assistant", "content": [tool_block]},   # echo the call
                {
                    "role": "user",
                    "content": [{
                        "type": "tool_result",
                        "tool_use_id": tool_block.id,
                        "content": tool_result,
                    }],
                },
            ],
            temperature=0.3,
            max_tokens=500,
        )

        print('\n')
        print("OFFICIAL RESULT OF CHATBOT")
        return plaintext(follow)

    # 5️⃣  No tool needed – return the first answer  ────────────────────────
    return plaintext(first)


In [109]:
q = "Which three universities in Texas have the lowest total cost of attendance?"

print(generate_response_with_rag(q))



REASONING GENRATED BY THE AGENT: '
' I identified the three Texas universities with the lowest total cost of attendance by summing books/supplies, room/board, and other on-campus expenses.


CODE GENRATED BY THE AGENT: '
'  # Filter for Texas universities
tx_universities = df[df['STABBR'] == 'TX']

# Calculate total cost of attendance (on-campus)
tx_universities['total_cost'] = tx_universities['BOOKSUPPLY'] + tx_universities['ROOMBOARD_ON'] + tx_universities['OTHEREXPENSE_ON']

# Sort by total cost and get the three lowest
lowest_cost = tx_universities.nsmallest(3, 'total_cost')

# Create the answer string
answer = f"The three Texas universities with lowest total cost are: {', '.join(lowest_cost['INSTNM'].tolist())}, ranging from ${lowest_cost['total_cost'].min():.2f} to ${lowest_cost['total_cost'].max():.2f}."

# Create a table with relevant information
table = lowest_cost[['INSTNM', 'CITY', 'BOOKSUPPLY', 'ROOMBOARD_ON', 'OTHEREXPENSE_ON', 'total_cost']].reset_index(drop=True)


<string>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy




OFFICIAL RESULT OF CHATBOT
**1. How I answered**
I needed to find universities in Texas with the lowest total costs, so I decided to calculate the total cost of attendance by summing the relevant expense categories. After filtering for Texas institutions, I sorted them by this total cost to identify the three most affordable options.

**2. Code in plain English**
The code filters the dataset to include only universities in Texas, then calculates the total cost of attendance by adding books/supplies, on-campus room/board, and other on-campus expenses. It sorts these universities by total cost in ascending order and selects the top three with the lowest costs.

**3. Result**
The three Texas universities with lowest total cost are: Howard College, University of North Texas, Texas Tech University, ranging from $12155.00 to $16170.00.

| INSTNM                    | CITY       |   BOOKSUPPLY |   ROOMBOARD_ON |   OTHEREXPENSE_ON |   total_cost |
|:--------------------------|:-----------|---

Play with your own questions. The agent automatically:
1. Figures out the numeric intent.
2. Writes a Pandas snippet, runs it, captures the result.
3. Composes a natural‑language answer.

Because the data are mostly numeric, embeddings aren’t needed; we keep everything in memory for precise filtering.